In [7]:
search_for = 81
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.00443267822265625
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 70
all cases: 75913222
type: [1, 1, 1, 1, 81] 81
cases of this type: 43046721
10000 12.467878074267825
20000 12.989996658101466
30000 50.28092250644183
40000 39.843524685114375
50000 40.86613435569409
60000 40.99059451761629
70000 41.82822676979085
80000 40.10993891333694
90000 39.345424416554884
100000 39.02389731964691
110000 41.04744864084284
120000 38.502534853631936
130000 43.0460906020689
140000 51.2419525699756
150000 39.41918625828684
160000 41.97363844988469
170000 39.28265866494339
180000 36.5174481901408
190000 36.852247332641724
200000 41.81309426944508
210000 37.58672558117112
220000 37.52597931542313
230000 41.23137402532298
240000 36.25273045959139
250000 43.317953537561095
260000 6.741475417096389
270000 6.970158660666506
280000 7.991508027699511
290000 40.489889572879235
300000 37.794934625608455
310000 36.81050168593573
32000

3080000 30.284186010346044
3090000 30.631909526265577
3100000 30.395871146337747
3110000 31.58775959818382
3120000 31.278315884054308
3130000 32.59931378987022
3140000 28.59565767143647
3150000 14.350924116307736
3160000 14.720059178140282
3170000 16.694138925744507
3180000 37.2554253197413
3190000 34.33216300832307
3200000 34.3445182132902
3210000 32.76953812584615
3220000 25.179787456374445
3230000 12.273906818299691
3240000 12.144514974293946
3250000 18.03506699328502
3260000 34.7803697149166
3270000 31.557633105736972
3280000 32.5248465836631
3290000 31.751332692954357
3300000 31.460354820824012
3310000 31.79927530970347
3320000 31.801702603165015
3330000 30.743615586901928
3340000 32.61324820077829
3350000 34.22267789005705
3360000 31.439362238366844
3370000 32.72590290790021
3380000 31.240645062472723
3390000 29.45948692385753
3400000 32.24254464858632
3410000 31.304117885269736
3420000 31.00410587718466
3430000 29.685964254706818
3440000 28.809368433991157
3450000 6.091201816067

6180000 25.865156387967218
6190000 28.424944722352187
6200000 28.625769765096216
6210000 31.882134926863905
6220000 30.56223133917785
6230000 29.25150820735956
6240000 29.724269129005382
6250000 29.923053528573515
6260000 32.46161580027384
6270000 29.988400041783322
6280000 32.1476628494424
6290000 26.482688768321836
6300000 14.20980403789936
6310000 13.950388650399647
6320000 18.15584202522724
6330000 36.4481107327709
6340000 30.16328149762058
6350000 29.957450557195727
6360000 29.928486377019865
6370000 32.13594332503859
6380000 34.48944552004765
6390000 32.967943437552194
6400000 30.571761562207975
6410000 30.355254282045603
6420000 30.517365480603015
6430000 30.426026015220284
6440000 32.71599519627799
6450000 20.42594552159724
6460000 11.757020206172704
6470000 12.023990616136789
6480000 22.746010996239768
6490000 35.137864085974414
6500000 29.55331623090788
6510000 31.376941022868127
6520000 28.59675081391454
6530000 29.8461164779575
6540000 29.478904937141337
6550000 32.67098683

9270000 22.859482360125753
9280000 29.558682111649198
9290000 14.840554444121505
9300000 6.043505170682973
9310000 5.641902604945422
9320000 14.418707413406585
9330000 27.593564514719183
9340000 23.901038450514793
9350000 25.42303538045437
9360000 25.067847737279124
9370000 26.469163651880862
9380000 25.345183228346958
9390000 24.708741472629704
9400000 23.255814992998598
9410000 26.863980213856376
9420000 25.62828075017602
9430000 27.91327081007886
9440000 20.249889150747645
9450000 12.131626931579286
9460000 12.238299596966268
9470000 16.097834769557302
9480000 25.426101484684214
9490000 22.980882117636085
9500000 28.424774113309027
9510000 30.221617819351565
9520000 26.909530443245888
9530000 26.673188563046338
9540000 26.120475911431907
9550000 27.175902216180162
9560000 27.653494541456027
9570000 28.06168406489094
9580000 27.891074016731146
9590000 29.28821172885308
9600000 25.845596094469492
9610000 25.45364225926431
9620000 23.145546270908078
9630000 28.433824919582875
9640000 2

12270000 28.53356139798591
12280000 28.441157156780363
12290000 28.63384656682605
12300000 25.466549556211273
12310000 27.02921089607076
12320000 25.573479466372287
12330000 25.287110036214433
12340000 24.31400932625703
12350000 25.143699517722396
12360000 25.85253643045003
12370000 12.56058297766483
12380000 9.273022111097177
12390000 9.788809927977839
12400000 20.010566555888296
12410000 25.4353904991745
12420000 24.965452060171827
12430000 25.119461107495862
12440000 26.692890164114687
12450000 23.131424764931985
12460000 24.465250944954157
12470000 26.290341031687433
12480000 10.768794005041824
12490000 5.504520679302574
12500000 5.324224726144684
12510000 17.305113323692666
12520000 27.850769359742205
12530000 21.8658420132097
12540000 26.55736110769692
12550000 26.301819464431283
12560000 26.285709029634955
12570000 22.911875489618488
12580000 24.930120503723224
12590000 17.871842512434483
12600000 12.902576924162586
12610000 10.52490588295138
12620000 18.667191594665834
12630000

...

62910000 1.5859650683580904
62920000 1.1801663718933264
62930000 1.9969441878052818
62940000 1.8063483860146339
62950000 2.25555295814693
62960000 1.465990783472167
62970000 2.752414745685458
62980000 2.584281327001254
62990000 1.9794639866174593
63000000 2.2453669599858523
63010000 1.7166615450145803
63020000 1.516611664466911
63030000 1.580847307836612
63040000 1.2510466840142014
63050000 1.6227129885779223
63060000 1.6897158607852856
63070000 3.0361438252031405
63080000 1.7157892016336123
63090000 2.228497788586259
63100000 1.6919195755203567
63110000 2.841166383761936
63120000 1.2464990817204051
63130000 1.7641570818760397
63140000 1.346224586174806
63150000 1.7440437306175098
63160000 1.2337816731574138
63170000 1.9342083820082345
63180000 1.578009699885368
63190000 2.2053201439012686
63200000 2.383518970929464
63210000 2.4285873943095737
63220000 1.3930845523767472
63230000 1.7632237807271745
63240000 1.2749077214637332
63250000 1.691137422980428
63260000 1.5977812879110045
63270

65810000 1.5275536688645153
65820000 1.3326120139987072
65830000 1.110748835120201
65840000 1.1794620899317796
65850000 1.3050335956340233
65860000 1.2919977997221548
65870000 1.227989955114391
65880000 1.1491403363575405
65890000 1.0084192844918567
65900000 1.2146668472185
65910000 1.1394847679710651
65920000 1.2792277299102546
65930000 1.118337544989864
65940000 1.7098364451785353
65950000 1.3518362883743047
65960000 1.3209597218594948
65970000 1.2986346633284225
65980000 1.5214457883084218
65990000 1.2688922885393301
66000000 1.3736142212477127
66010000 1.3108451769322156
66020000 1.3155971461957563
66030000 1.3532611733248234
66040000 1.3186988039386671
66050000 1.2102167104319732
66060000 1.3009450806030034
66070000 1.4183354459364412
66080000 1.4744618873408768
66090000 1.889783295026117
66100000 1.6842222844458818
66110000 1.9462433222787248
66120000 1.6639930108491845
66130000 2.3245689845614037
66140000 2.1970976259475017
66150000 2.8229230011371245
66160000 2.3647606432229678

68760000 1.7208118799262047
68770000 1.5732388973611988
68780000 1.5777804776397544
68790000 1.5558057159129646
68800000 1.5237692651673556
68810000 1.570989701611148
68820000 1.6537541654458443
68830000 1.821442616510272
68840000 1.6582050519565341
68850000 1.5733651280427907
68860000 1.5381644402110577
68870000 1.5632122519933647
68880000 1.5625345669723352
68890000 1.5207205743008851
68900000 1.66332280236423
68910000 1.654977946800775
68920000 2.0426246858753365
68930000 1.9349166514155336
68940000 1.7539868480003544
68950000 1.691257980175495
68960000 1.6497353856403698
68970000 1.7159272174894944
68980000 1.6447279599770308
68990000 1.7998839166196319
69000000 1.9345116578155357
69010000 2.0656085054097573
69020000 1.7613062749532462
69030000 1.6824445207630794
69040000 1.6699783034750622
69050000 1.7453804774845045
69060000 1.6847324168885283
69070000 1.757083726518154
69080000 1.8688893312433825
best so far: 0
type: [1, 1, 3, 3, 9] 81
cases of this type: 1594323
69090000 1.6552

71600000 0.36133441903061336
71610000 0.3569174217474196
71620000 0.352067848560373
71630000 0.35140006570862403
71640000 0.3516575619433059
71650000 0.3499869251614014
71660000 0.35114117535327544
71670000 0.349100382789082
71680000 0.34902536538147927
71690000 0.34766993322061174
71700000 0.3458254750248591
71710000 0.3480905456886292
71720000 0.34637101288257705
71730000 0.3471522014095651
71740000 0.3477486834412813
71750000 0.3463718169964684
71760000 0.34091172732803554
71770000 0.35853227577269076
71780000 0.34763085646665093
71790000 0.34099120027222896
71800000 0.33928731735161943
71810000 0.6976531370958885
71820000 0.8810412125031949
71830000 0.8683196345775126
71840000 0.8660294438234966
71850000 0.873052062268138
71860000 0.8558204998639821
71870000 0.8912588457778031
71880000 0.8506709549600019
71890000 0.8752050762004455
71900000 0.8265192511202893
71910000 0.862424275343471
71920000 0.8246410451855263
best so far: 0
type: [1, 1, 9, 3, 3] 81
cases of this type: 177147
71

74350000 0.14489284866384664
74360000 0.19536185552027488
best so far: 0
type: [1, 3, 1, 9, 3] 81
cases of this type: 177147
74370000 0.24723789004303348
74380000 0.20568535898816587
74390000 0.19905442618220381
74400000 0.19991828200980025
74410000 0.18983283221244812
74420000 0.2006723786724408
74430000 0.1925882189180189
74440000 0.1940897699869076
74450000 0.1938468638714022
74460000 0.18377403432457978
74470000 0.1916567029337883
74480000 0.18528572178255187
74490000 0.19186637318970098
74500000 0.18385591230718293
74510000 0.18557842959617243
74520000 0.1828016543061336
74530000 0.17845615089809896
74540000 0.18452374508407382
best so far: 0
type: [1, 3, 1, 27, 1] 81
cases of this type: 59049
74550000 0.14390912391764588
74560000 0.0983019648897648
74570000 0.09706860832340188
74580000 0.09667581762637033
74590000 0.09539170920952161
74600000 0.09444315714508957
best so far: 0
type: [1, 3, 3, 1, 9] 81
cases of this type: 177147
74610000 0.15587681905380887
74620000 0.217363818604